In [1]:
# Cell 1: 导入库与配置 MPS 加速
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from collections import Counter
import os

# 1. 设置设备 (关键：使用 Mac M4 的 MPS 加速)
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"🚀 当前运行设备: {device}")

# 2. 读取数据
filename = 'processed_data.csv'
# 自动寻找路径
if os.path.exists(os.path.join('..', 'data', filename)):
    df = pd.read_csv(os.path.join('..', 'data', filename))
elif os.path.exists(os.path.join('data', filename)):
    df = pd.read_csv(os.path.join('data', filename))
else:
    raise FileNotFoundError("找不到 processed_data.csv")

# 过滤掉空值
df = df.dropna(subset=['cleaned_text', 'labels'])

# 3. 标签编码 (把 good/bad/neutral 变成 0, 1, 2)
label_map = {'bad': 0, 'neutral': 1, 'good': 2}
df['label_id'] = df['labels'].map(label_map)

print(f"✅ 数据准备就绪，共 {len(df)} 条")
display(df.head(2))

🚀 当前运行设备: mps
✅ 数据准备就绪，共 219286 条


,Unnamed: 0,tweets,labels,cleaned_text,label_id
0,0,ChatGPT: Optimizing Language Models for Dialog...,neutral,chatgpt optimizing language models for dialogue,1
1,1,"Try talking with ChatGPT, our new AI system wh...",good,try talking with chatgpt our new ai system whi...,2


In [2]:
# Cell 2: 构建词表与 Dataset 类
# 这是一个手动实现的简单 Tokenizer，比调用 BERT 库更直观，适合展示原理

class TextDataset(Dataset):
    def __init__(self, texts, labels, vocab=None, max_len=50):
        self.texts = texts
        self.labels = labels
        self.max_len = max_len
        
        # 如果没传词表，就根据当前数据建立一个
        if vocab is None:
            all_words = " ".join(texts).split()
            word_counts = Counter(all_words)
            # 只保留出现超过 2 次的词，减少噪音
            vocab = {word: i+2 for i, (word, count) in enumerate(word_counts.items()) if count > 2}
            vocab['<PAD>'] = 0 # 填充位
            vocab['<UNK>'] = 1 # 未知词
            print(f"📚 词表构建完成，包含 {len(vocab)} 个单词")
        
        self.vocab = vocab

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts.iloc[idx]
        label = self.labels.iloc[idx]
        
        # 文本转数字序列
        tokens = [self.vocab.get(word, 1) for word in text.split()]
        
        # 截断或填充 (Padding) 保证长度一致
        if len(tokens) < self.max_len:
            tokens = tokens + [0] * (self.max_len - len(tokens))
        else:
            tokens = tokens[:self.max_len]
            
        return torch.tensor(tokens, dtype=torch.long), torch.tensor(label, dtype=torch.long)

# 划分数据集
X_train, X_test, y_train, y_test = train_test_split(df['cleaned_text'], df['label_id'], test_size=0.2, random_state=42)

# 创建 Dataset 实例
train_dataset = TextDataset(X_train, y_train)
test_dataset = TextDataset(X_test, y_test, vocab=train_dataset.vocab) # 测试集必须用训练集的词表！

# 创建 DataLoader (批量搬运工)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

print("✅ PyTorch 数据管道搭建完成！")

📚 词表构建完成，包含 27506 个单词
✅ PyTorch 数据管道搭建完成！


In [3]:
# Cell 3: 定义 LSTM 模型与训练循环

class RNNClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim=100, hidden_dim=128, num_classes=3):
        super(RNNClassifier, self).__init__()
        # 1. 词嵌入层：把数字ID变成向量
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        # 2. LSTM 层：捕捉序列特征
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        # 3. 全连接层：输出分类结果
        self.fc = nn.Linear(hidden_dim, num_classes)
        
    def forward(self, x):
        x = self.embedding(x)
        # LSTM 返回: output, (hidden_state, cell_state)
        # 我们只需要最后一步的 hidden_state
        _, (h_n, _) = self.lstm(x)
        out = self.fc(h_n[-1]) # 取最后一层的状态
        return out

# 初始化模型
vocab_size = len(train_dataset.vocab)
model = RNNClassifier(vocab_size).to(device) # 把模型搬到 GPU 上

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# === 训练循环 ===
print("🔥 开始训练 RNN 模型 (使用 MPS 加速)...")
epochs = 5

for epoch in range(epochs):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    for inputs, labels in train_loader:
        # 搬运数据到 GPU
        inputs, labels = inputs.to(device), labels.to(device)
        
        # 前向传播
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # 反向传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss/len(train_loader):.4f}, Train Acc: {100 * correct / total:.2f}%")

print("🎉 训练完成！")

🔥 开始训练 RNN 模型 (使用 MPS 加速)...
Epoch [1/5], Loss: 0.6369, Train Acc: 73.94%
Epoch [2/5], Loss: 0.3625, Train Acc: 87.41%
Epoch [3/5], Loss: 0.2833, Train Acc: 90.55%
Epoch [4/5], Loss: 0.2422, Train Acc: 92.03%
Epoch [5/5], Loss: 0.2109, Train Acc: 93.12%
🎉 训练完成！


In [4]:
# Cell 4: 评估模型
from sklearn.metrics import classification_report

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

print("\n🏆 RNN 模型评估报告:")
# 还原标签名
target_names = ['bad', 'neutral', 'good'] # 对应 0, 1, 2
print(classification_report(all_labels, all_preds, target_names=target_names))


🏆 RNN 模型评估报告:
              precision    recall  f1-score   support

         bad       0.95      0.94      0.95     21504
     neutral       0.82      0.82      0.82     11075
        good       0.90      0.90      0.90     11279

    accuracy                           0.90     43858
   macro avg       0.89      0.89      0.89     43858
weighted avg       0.90      0.90      0.90     43858

